In [1]:
from feos import *
from feos.si import *
from feos.eos import *
from feos.pcsaft import *
import numpy as np
import pandas as pd

## Parameter

In [13]:
parameters_2b = PcSaftParameters.from_json(['water_2B'], 'rehner2020.json')
parameters_3b = PcSaftParameters.from_json(['water_3B'], 'rehner2020.json')
parameters_4c = PcSaftParameters.from_json(['water_4C'], 'rehner2020.json')

## Zustandsgleichung

In [3]:
eos_2b = EquationOfState.pcsaft(parameters_2b)
eos_3b = EquationOfState.pcsaft(parameters_3b)
eos_4c = EquationOfState.pcsaft(parameters_4c)

## Thermodynamischer Zustand

### 2B

In [4]:
parameters_2b

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|water_2B|18.01528|1|2.937523956051823|272.02757407828676|0|0|0.04448012165716923|3125.3202766200056|1|1|

In [5]:
s_2b = State(eos_2b, temperature=298.1*KELVIN, pressure=1*BAR)
s_2b

|temperature|density|
|-|-|
|298.10000 K|55.46388 kmol/m³|

In [6]:
for name, value in s_2b.helmholtz_energy_contributions():
    print(f"{name:<20}\t: {value * NAV / JOULE * MOL:<15.6f} J/mol")

Ideal gas (QSPR)    	: 15431.365606    J/mol
Hard Sphere         	: 9019.023404     J/mol
Hard Chain          	: 0.000000        J/mol
Dispersion          	: -13932.565538   J/mol
Association         	: -19058.994923   J/mol


### 3B

In [7]:
parameters_3b

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|water_3B|18.01528|1.6330279759286626|2.4570046159550527|238.31794591948915|0|0|0.037807044304069184|2749.004567423258|2|1|

In [8]:
s_3b = State(eos_3b, temperature=298.1*KELVIN, pressure=1*BAR)
s_3b

|temperature|density|
|-|-|
|298.10000 K|55.89010 kmol/m³|

In [9]:
for name, value in s_3b.helmholtz_energy_contributions():
    print(f"{name:<20}\t: {value * NAV / JOULE * MOL:<15.6f} J/mol")

Ideal gas (QSPR)    	: 15450.339682    J/mol
Hard Sphere         	: 13398.315169    J/mol
Hard Chain          	: -2145.040856    J/mol
Dispersion          	: -17921.694693   J/mol
Association         	: -17365.878605   J/mol


### 4C

In [10]:
parameters_4c

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|water_4C|18.01528|1.866828208262284|2.395032022200541|169.7751872692369|0|0|0.13373757842938191|1772.0393059972052|2|2|

In [11]:
s_4c = State(eos_4c, temperature=298.1*KELVIN, pressure=1*BAR)
s_4c

|temperature|density|
|-|-|
|298.10000 K|55.21791 kmol/m³|

In [12]:
for name, value in s_4c.helmholtz_energy_contributions():
    print(f"{name:<20}\t: {value * NAV / JOULE * MOL:<15.6f} J/mol")

Ideal gas (QSPR)    	: 15420.349496    J/mol
Hard Sphere         	: 15683.846063    J/mol
Hard Chain          	: -2985.650102    J/mol
Dispersion          	: -15017.782883   J/mol
Association         	: -21677.693979   J/mol


In [13]:
s_4c.compressibility(), s_4c.pressure() / s_4c.density / RGAS / s_4c.temperature

(0.0007306743801645198, 0.0007306743801645197)

In [14]:
np.rad2deg(np.sin(109.5 / 2 * np.pi / 180))

46.790114485764214

# Polymer + Wasser

In [18]:
# we need to rebuild the EOS for each temperature since
# sigma and k_ij are functions of temperature
def equation_of_state(temperature, mw_monomer, mw_linker, phi, y, verbose=False):
    """Build equation of state for given temperature in Kelvin."""
    
    # water
    # na = nb = 1 (2B association)
    sigma_water = 2.7927 + 10.11 * np.exp(-0.01775 * temperature) - 1.417 * np.exp(-0.01146 * temperature)
    water = PureRecord(
        Identifier(name='water'),
        molarweight=18.01528,
        model_record=PcSaftRecord(m=1.2047, sigma=sigma_water, epsilon_k=353.95, epsilon_k_ab=2425.67, kappa_ab=0.0451, na=1, nb=1)
    )
    
    # polymer
    n_linker = y / (1 - y) * GRAM / mw_monomer
    n_polymer = phi / 2.0 * n_linker
    mw_polymer = (GRAM + n_linker * mw_linker) / n_polymer
    m = mw_polymer / mw_monomer
    
    na = nb = 90
    polymer = PureRecord(
        Identifier(name='polymer'),
        molarweight=mw_polymer / GRAM * MOL,
        model_record=PcSaftRecord(m=m, sigma=5.38, epsilon_k=297.343, epsilon_k_ab=175, kappa_ab=0.045, na=na, nb=nb)
    )
    
    # binary interaction parameter
    k_ij = -0.17 + 0.0013 * (temperature - 298.15)
    
    # polymer index: 0
    # water index: 1
    parameters = PcSaftParameters.new_binary([polymer, water], k_ij)
    
    # elastic parameters
    eos_pure = EquationOfState.pcsaft(PcSaftParameters.new_pure(polymer))
    dry_polymer_density = State(eos_pure, temperature=temperature*KELVIN, pressure=1.0135*BAR).density
    v0 = n_polymer / ANGSTROM**3 / dry_polymer_density
    elastic = ElasticParameters(n_linker * NAV, n_polymer * NAV, v0, phi)
    
    if verbose:
        print(f"n_linker: {n_linker / MOL} mol")
        print(f"n_polymer: {n_polymer / MOL} mol")
        print(f"mw_polymer: {mw_polymer / GRAM * MOL} g/mol")
        print("m", m)
        print(f"v0: {v0} A³")
        
    return EquationOfState.pcsaft(parameters, elastic=elastic)

In [19]:
def equation_of_state_pp():
    """Build equation of state for given temperature in Kelvin."""
   
    # polymer
    # na = nb = 1 per monomer
    m = 89.60932075
    na = nb = 90
    mw = 10139.29464
    polymer = PureRecord(
        Identifier(name='polymer'),
        molarweight=mw,
        model_record=PcSaftRecord(m=m, sigma=5.38, epsilon_k=297.343, epsilon_k_ab=175, kappa_ab=0.045, na=na, nb=nb)
    )    
    # polymer index: 0
    # water index: 1
    parameters = PcSaftParameters.new_binary([polymer])
    return EquationOfState.pcsaft(parameters)

In [20]:
temperatures = np.array([
    317.8510029,
    313.1232092,
    313.1232092,
    306.6762178,
    305.9598854,
    308.3954155,
    308.1088825,
    305.9598854,
    305.8166189,
    305.1002865,
    304.6704871,
    304.3839542,
    302.8080229,
    302.9512894,
    302.52149,
    297.7936963,
    297.6504298,
    293.0659026,
    292.7793696
])

x_polymer = np.array([
    0.00150408209788188,
    0.00150408209788188,
    0.00126982511519473,
    0.00207952783043735,
    0.00020423279576647,
    0.00207952783043735,
    0.00150408209788188,
    0.000226975457389918,
    0.000175635327192893,
    0.000155532334683888,
    0.000143236350040674,
    0.000134939067563479,
    0.000100754335021515,
    0.000102657259483328,
    9.43421080566932E-05,
    7.317410522015E-05,
    7.2523209651888E-05,
    5.99843098952374E-05,
    5.91144720479721E-05
])

densities = np.array([
    31622.1475685248,
    31700.498450108,
    33956.3399012912,
    27334.1991995267,
    50225.3892494127,
    27310.3908429378,
    31779.4212551547,
    49724.5562178088,
    50872.0802808432,
    51347.5490495209,
    51642.2525764164,
    51842.8165327244,
    52691.0971146765,
    52642.3010022701,
    52852.8084682972,
    53449.9747942836,
    53468.2760121439,
    53841.8681872705,
    53867.0890750594
])

In [21]:
# check single state
temperature = temperatures[0] * KELVIN
x = np.array([x_polymer[0], 1-x_polymer[0]])
density = densities[0] * MOL / METER**3

mw_monomer = 113.15 * GRAM / MOL
mw_linker = 154.16 * GRAM / MOL
phi = 2.24
y = 0.01

eos = equation_of_state(temperature / KELVIN, mw_monomer, mw_linker, phi, y, verbose=True) # only use numeric value - without unit

n_linker: 8.927096863464517e-05 mol
n_polymer: 9.99834848708026e-05 mol
mw_polymer: 10139.294642857143 g/mol
m 89.60932074995264
v0: 9.140769953403274e+23 A³


# All states

In [22]:
%time
mw_monomer = 113.15 * GRAM / MOL
mw_linker = 154.16 * GRAM / MOL
phi = 2.24
y = 0.01

results = []
for ti, xi, rhoi in zip(temperatures * KELVIN, x_polymer, densities * MOL / METER**3):
    eos = equation_of_state(ti / KELVIN, mw_monomer, mw_linker, phi, y)
    state = State(eos, ti, density=rhoi, molefracs=np.array([xi, 1-xi]))
    contributions = state.helmholtz_energy_contributions()
    ln_phi = state.ln_phi()
    results.append({
        "temperature [K]": ti / KELVIN, 
        "x_polymer": xi,
        "density [mol/m³]": rhoi / MOL * METER**3,
        "A_assoc [J/mol]": contributions[-2][1] * NAV / JOULE * MOL,
        "Z_assoc": state.pressure_contributions()[-2][1] / state.density / RGAS / state.temperature,
        "A_elastic [J/mol]": contributions[-1][1] * NAV / JOULE * MOL,
        "Z_elastic": state.pressure_contributions()[-1][1] / state.density / RGAS / state.temperature,
        "p [bar]": state.pressure() / BAR,
        "ln_phi": ln_phi
    })

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 10.3 µs


In [23]:
res = pd.DataFrame(results)
res

,temperature [K],x_polymer,density [mol/m³],A_assoc [J/mol],Z_assoc,A_elastic [J/mol],Z_elastic,p [bar],ln_phi
0,317.851003,0.001504,31622.147569,-11712.416546,-3.106488,0.121314,-0.000121,1.114049,"[-314.64878788281004, -2.46852697271931]"
1,313.123209,0.001504,31700.498450,-11849.592916,-3.138008,0.120015,-0.000121,1.113040,"[-325.20983117644585, -2.7191498527119875]"
2,313.123209,0.001270,33956.339901,-11976.132591,-3.119271,0.130324,-0.000119,1.118111,"[-324.3978720325559, -2.7248182997165107]"
3,306.676218,0.002080,27334.199200,-11772.546939,-3.227157,0.099021,-0.000126,1.100416,"[-340.071898734769, -3.065242270445012]"
4,305.959885,0.000204,50225.389249,-12922.165221,-3.061159,0.194948,-0.000087,1.123528,"[-342.01744689021587, -3.1259450757532115]"
5,308.395416,0.002080,27310.390843,-11720.808215,-3.215073,0.099362,-0.000126,1.100837,"[-336.34071057367976, -2.968102214309463]"
6,308.108882,0.001504,31779.421255,-11995.700847,-3.171170,0.118694,-0.000122,1.111936,"[-336.9246498308432, -2.9943514194349734]"
7,305.959885,0.000227,49724.556218,-12902.206595,-3.063696,0.194113,-0.000089,1.125067,"[-342.0559917008422, -3.1273045172465723]"
8,305.816619,0.000176,50872.080281,-12951.366901,-3.058705,0.195430,-0.000084,1.121135,"[-342.53951023094794, -3.1318363648056735]"
9,305.100286,0.000156,51347.549050,-12988.243411,-3.060234,0.194983,-0.000082,1.118886,"[-345.4298675422592, -3.170014516288221]"


In [24]:
res["Z_elastic"].values

array([-1.21139304e-04, -1.21441502e-04, -1.19203753e-04, -1.25740059e-04,
       -8.69080649e-05, -1.25585448e-04, -1.21805842e-04, -8.89756548e-05,
       -8.39767781e-05, -8.16581310e-05, -8.00987779e-05, -7.89760862e-05,
       -7.36032459e-05, -7.39394136e-05, -7.24225345e-05, -6.81007158e-05,
       -6.79517468e-05, -6.49048345e-05, -6.46734451e-05])